In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os
import inspect
import getpass

import pandas as pd
import matplotlib.pyplot as plt



project_path = os.path.abspath( os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe()))) + "/../" )
sys.path.insert(0, project_path)
print(project_path)

from database_storage.helper import *

from database_storage.database.database import *
from database_storage.main import *

from database_storage.objects.human import *
from database_storage.objects.issue import *
from database_storage.objects.kpi import *
from database_storage.objects.machine import *
from database_storage.objects.tag import *
from database_storage.objects.maintenanceworkorder import *


/Users/sam11/Git/ml-py


## Connect to Database

In [2]:
databaseSchema_file = os.path.join(project_path ,'database_storage','database', 'DatabaseSchema.yaml')
databaseSchema_dict = openYAMLFile(databaseSchema_file)

yaml file open


In [3]:
#user = getpass.getuser()
password = getpass.getpass("Password:")

user = "neo4j"
#password = 'GREYSTONE!!'

database = DatabaseNeo4J("bolt://127.0.0.1:7687", user, password, databaseSchema_dict)

Password:········


### get CSV information

In [4]:
csv_file = os.path.join(project_path ,'data', 'public', 'READABLE_TAGS.csv')
#csv_file = os.path.join(project_path ,'data', 'test.csv')

dataframe = pd.read_csv(csv_file)
dataframe.fillna("", inplace=True)

#csvSchemas_file = os.path.join(project_path , 'data', 'testCSVheader.yaml')
csvSchemas_file = os.path.join(project_path , 'data',  'public', 'mine_header.yaml')

csvSchemas_dict = openYAMLFile(csvSchemas_file)

yaml file open


### Clean the database

In [5]:
database.deleteData()
database.dropIndexes()
database.createIndexes()

1

## Store the data

In [6]:
done = graphDatabase_from_TaggedCSV(database, dataframe, csvSchemas_dict)

100%|██████████| 5485/5485 [07:49<00:00, 11.69it/s]


# KPI

## Create the object for the KPI

In [27]:
kpiDict = {
    'issue':{
            'description_problem': ["_"],
            'description_solution': None,
            'description_cause': None,
            'description_effect': None,
            'machine_down': None,
            'necessary_part': None,
            'part_in_process': None,
            'cost': [214, "378", "_"],

            'date_machine_down': None,
            'date_workorder_start': None,
            'date_maintenance_technician_arrive': None,
            'date_problem_found': None,
            'date_part_ordered': None,
            'date_part_received': None,
            'date_problem_solve': None,
            'date_machine_up': None,
            'date_workorder_completion': None
        },
    'human':{
        'name': None
    },
    'operator': {
        'name': None
    },
    'technician': {
        'name': None,
        'skills': None,
        'crafts': None
    },
    'machine': {
        'name': None,
        'manufacturer': None,
        'location': None
    },
    'machine_type': {
        'type': None
    },
    'tag': {
        'keyword': None,
        'synonyms':  None
    },
    'tag_oneGram': {
        'keyword':  None,
        'synonyms':  None
    },
    'tag_nGram': {
        'keyword':  None,
        'synonyms':  None
    },
    'tag_item': {
        'keyword':  None,
        'synonyms':  None
    },
    'tag_itemAsProblem': {
        'keyword':  None,
        'synonyms':  None
    },
    'tag_itemAsSolution': {
        'keyword':  None,
        'synonyms':  None
    },
    'tag_problem': {
        'keyword':  None,
        'synonyms':  None
    },
    'tag_solution': {
        'keyword':  None,
        'synonyms':  None
    },
    'tag_unknown': {
        'keyword':  None,
        'synonyms':  None
    },
    'tag_poblemItem': {
        'keyword':  None,
        'synonyms':  None
    },
    'tag_solutionItem': {
        'keyword':  None,
        'synonyms':  None
    }
    }
kpi = Kpi(database, kpiDict)
kpi.createObjectsFromDict()
print(kpi.tag_item)

object  =  <class 'database_storage.objects.tag.TagItem'>
	keyword  =  None
	synonyms  =  None
	similarTo  =  None
	children  =  None


## get the hierarchy of your item

In [ ]:
g = kpi.tree_itemsHierarchie(child= True, parent=True)

In [ ]:
plt.subplot(121)
nx.draw(g[0][0], with_labels=True, font_weight='bold')
plt.subplot(122)
nx.draw(g[0][1], with_labels=True, font_weight='bold')

## Create the query to filter the dataframe

In [28]:
filterQuery = kpi.cypher_filterDatabase()
done, filterResult = database.runQuery(filterQuery)

dataframe = resultToObservationDataframe(filterResult)
dataframe

8it [00:00, 22339.83it/s]

issue


,issue.description_of_problem,issue.cost
0,Replace broken adaptor,378
1,REPLACE WEAR PACKAGE LOWER WALL,214
2,air con. not getting cold.,214
3,REPLACE HP SEAL AT DISCHARGE,214
4,horn not working,214
5,FABRICATE NEW ACCESS STEP,214
6,A/C REPAIRS,214
7,air con. not getting cold.,214


In [23]:
filterQuery

'MATCH (issue:ISSUE)\nWHERE issue.cost = 214 OR issue.cost = 378\nRETURN issue.description_of_problem'

In [22]:
print(kpi.issue)

object  =  <class 'database_storage.objects.issue.Issue'>
	problem  =  ['_']
	solution  =  None
	cause  =  None
	effects  =  None
	part_in_process  =  None
	necessary_part  =  None
	machine_down  =  None
	cost  =  [214, 378]
	date_machine_up  =  None
	date_machine_down  =  None
	date_workorder_start  =  None
	date_workorder_completion  =  None
	date_maintenance_technician_arrive  =  None
	date_problem_found  =  None
	date_problem_solve  =  None
	date_part_ordered  =  None
	date_part_received  =  None
	time_to_repair  =  None
	time_work_order_completion  =  None
	time_to_dispatch  =  None
	time_to_return_to_operation  =  None
	time_to_issue_work_order  =  None
	time_to_travel  =  None
	time_to_solve_problem  =  None
	time_to_diagnose  =  None
	time_to_order  =  None
	time_lead_for_part  =  None
	time_to_fix  =  None
	time_to_turn_on  =  None
